In [7]:
from aidream_registration import constants
import aidream_registration.utils.cohort_utils as cu


In [8]:
from pathlib import Path
import shutil
import os

In [9]:
# the list of perfusion patients :
list_patients = cu.get_perfusion_patients()
print(fr"Number of patients: {len(list_patients)}")


Number of patients: 186


In [17]:
# the referential table :
df_ref = cu.get_referential_table(list_patients)
df_ref.head(5)


,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_reference,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_reference,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model
0,AIDREAM_1,MMI-PROB_140,201211519BP,yes,1,2.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes
1,AIDREAM_10,MMI-PROB_050,201002036RR,yes,1,2.0,AC_repATLAS native,RS,yes,yes,AC_repATLAS,RS,yes,yes
2,AIDREAM_100,MMI-PROB_139,201210738EE,yes,1,0.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,no
3,AIDREAM_102,MMI-PROB_085,201101726ZB,yes,1,2.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes
4,AIDREAM_103,MMI-PROB_166,201311898RR,yes,1,2.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes


In [6]:
# The directory of the processed data :

dir_parent_processed = constants.DIR_DEFAULT_HARD_DRIVE / "AIDREAM DATA" / "FINAL" / "REGISTERED ON ATLAS PRE_RT T1"
dict_processed = {
    "Affine": dir_parent_processed / "PROCESSED",
    "SyN": dir_parent_processed / "PROCESSED SyN"
}

for dir_processed in dict_processed.values():
    dir_processed.mkdir(parents=True, exist_ok=True)


In [10]:
# First, copy the pre_RT ATLAS MRI images to the Affine processed directory :
for i, patient in enumerate(list_patients):

    print(fr"Processing {patient} .........")
    stage = "pre_RT"

    for imaging in ["T1", "T1CE", "FLAIR"]:

        path_src = (constants.DIR_DEFAULT_HARD_DRIVE
                    / "AIDREAM DATA"
                    / "MRI DATA"
                    / "REGISTERED MRI BY PIPELINE"
                    / "pre_RT"
                    / "OUTPUT_DIR"
                    / patient
                    / "skullstripping"
                    / fr"image_n4_register_ss_{imaging.lower()}.nii.gz")
        assert path_src.exists(), f"File {path_src} does not exist"

        path_dst = dict_processed["Affine"] / patient / f"{patient}_{stage}_{imaging}.nii.gz"

        if not path_dst.exists():

            print(fr"Copying pre_RT {imaging} .........")
            path_dst.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy(path_src, path_dst)


Processing AIDREAM_1 .........
Copying pre_RT T1 .........
Copying pre_RT T1CE .........
Copying pre_RT FLAIR .........
Processing AIDREAM_2 .........
Copying pre_RT T1 .........
Copying pre_RT T1CE .........
Copying pre_RT FLAIR .........
Processing AIDREAM_3 .........
Copying pre_RT T1 .........
Copying pre_RT T1CE .........
Copying pre_RT FLAIR .........
Processing AIDREAM_4 .........
Copying pre_RT T1 .........
Copying pre_RT T1CE .........
Copying pre_RT FLAIR .........
Processing AIDREAM_6 .........
Copying pre_RT T1 .........
Copying pre_RT T1CE .........
Copying pre_RT FLAIR .........
Processing AIDREAM_7 .........
Copying pre_RT T1 .........
Copying pre_RT T1CE .........
Copying pre_RT FLAIR .........
Processing AIDREAM_9 .........
Copying pre_RT T1 .........
Copying pre_RT T1CE .........
Copying pre_RT FLAIR .........
Processing AIDREAM_10 .........
Copying pre_RT T1 .........
Copying pre_RT T1CE .........
Copying pre_RT FLAIR .........
Processing AIDREAM_11 .........
Copying

In [11]:
# Now copy the registered cercare images to the Affine processed directory :
for i, patient in enumerate(list_patients):

    print(fr"Processing {i+1}/{len(list_patients)} {patient} .........")
    for interpolator in constants.LIST_INTERPOLATORS:

        dir_src = (constants.DIR_DEFAULT_HARD_DRIVE
                   / "AIDREAM DATA"
                   / "CERCARE DATA"
                   / "REGISTERED CERCARE BY PADDING"
                   / patient
                   / interpolator)
        assert dir_src.exists(), f"File {dir_src} does not exist"

        dir_dst = dict_processed["Affine"] / patient / f"CERCARE_{interpolator}"

        if not dir_dst.exists():

            print(fr"copying CERCARE {interpolator} .........")
            dir_dst.parent.mkdir(exist_ok=True, parents=True)
            shutil.copytree(dir_src, dir_dst)


Processing 1/186 AIDREAM_1 .........
copying CERCARE linear .........
copying CERCARE bSpline .........
copying CERCARE lanczosWindowedSinc .........
copying CERCARE nearestNeighbor .........
copying CERCARE genericLabel .........
Processing 2/186 AIDREAM_2 .........
copying CERCARE linear .........
copying CERCARE bSpline .........
copying CERCARE lanczosWindowedSinc .........
copying CERCARE nearestNeighbor .........
copying CERCARE genericLabel .........
Processing 3/186 AIDREAM_3 .........
copying CERCARE linear .........
copying CERCARE bSpline .........
copying CERCARE lanczosWindowedSinc .........
copying CERCARE nearestNeighbor .........
copying CERCARE genericLabel .........
Processing 4/186 AIDREAM_4 .........
copying CERCARE linear .........
copying CERCARE bSpline .........
copying CERCARE lanczosWindowedSinc .........
copying CERCARE nearestNeighbor .........
copying CERCARE genericLabel .........
Processing 5/186 AIDREAM_6 .........
copying CERCARE linear .........
copyin

In [12]:
# Move the registered brainmask, from the CERCARE_genericLabel folder to the patient folder :
for i, patient in enumerate(list_patients):

    print(fr"Processing {i+1}/{len(list_patients)} {patient} .........")

    path_brainmask = dict_processed["Affine"] / patient / "CERCARE_genericLabel" / fr"{patient}_brainmask_registered_by_padding_genericLabel.nii.gz"
    assert path_brainmask.exists(), f"File {path_brainmask} does not exist"
    path_dst = dict_processed["Affine"] / patient / f"{patient}_cercare_brainmask.nii.gz"

    os.rename(path_brainmask, path_dst)


Processing 1/186 AIDREAM_1 .........
Processing 2/186 AIDREAM_2 .........
Processing 3/186 AIDREAM_3 .........
Processing 4/186 AIDREAM_4 .........
Processing 5/186 AIDREAM_6 .........
Processing 6/186 AIDREAM_7 .........
Processing 7/186 AIDREAM_9 .........
Processing 8/186 AIDREAM_10 .........
Processing 9/186 AIDREAM_11 .........
Processing 10/186 AIDREAM_12 .........
Processing 11/186 AIDREAM_13 .........
Processing 12/186 AIDREAM_14 .........
Processing 13/186 AIDREAM_15 .........
Processing 14/186 AIDREAM_16 .........
Processing 15/186 AIDREAM_17 .........
Processing 16/186 AIDREAM_19 .........
Processing 17/186 AIDREAM_20 .........
Processing 18/186 AIDREAM_21 .........
Processing 19/186 AIDREAM_22 .........
Processing 20/186 AIDREAM_24 .........
Processing 21/186 AIDREAM_26 .........
Processing 22/186 AIDREAM_28 .........
Processing 23/186 AIDREAM_31 .........
Processing 24/186 AIDREAM_32 .........
Processing 25/186 AIDREAM_33 .........
Processing 26/186 AIDREAM_34 .........
Pr

In [14]:
# Rename the Cercare maps :
for i, patient in enumerate(list_patients):

    print(fr"Processing {i+1}/{len(list_patients)} {patient} .........")

    for interpolator in constants.LIST_INTERPOLATORS:

        for biomarker in list(set(constants.LIST_CERCARE_MAPS) - {"brainmask"}):

            path_biomarker = dict_processed["Affine"] / patient / f"CERCARE_{interpolator}" / fr"{patient}_{biomarker}_registered_by_padding_{interpolator}.nii.gz"
            assert path_biomarker.exists(), f"File {path_biomarker} does not exist"

            path_dst = dict_processed["Affine"] / patient / f"CERCARE_{interpolator}" / f"{patient}_{biomarker}.nii.gz"
            os.rename(path_biomarker, path_dst)


Processing 1/186 AIDREAM_1 .........
Processing 2/186 AIDREAM_2 .........
Processing 3/186 AIDREAM_3 .........
Processing 4/186 AIDREAM_4 .........
Processing 5/186 AIDREAM_6 .........
Processing 6/186 AIDREAM_7 .........
Processing 7/186 AIDREAM_9 .........
Processing 8/186 AIDREAM_10 .........
Processing 9/186 AIDREAM_11 .........
Processing 10/186 AIDREAM_12 .........
Processing 11/186 AIDREAM_13 .........
Processing 12/186 AIDREAM_14 .........
Processing 13/186 AIDREAM_15 .........
Processing 14/186 AIDREAM_16 .........
Processing 15/186 AIDREAM_17 .........
Processing 16/186 AIDREAM_19 .........
Processing 17/186 AIDREAM_20 .........
Processing 18/186 AIDREAM_21 .........
Processing 19/186 AIDREAM_22 .........
Processing 20/186 AIDREAM_24 .........
Processing 21/186 AIDREAM_26 .........
Processing 22/186 AIDREAM_28 .........
Processing 23/186 AIDREAM_31 .........
Processing 24/186 AIDREAM_32 .........
Processing 25/186 AIDREAM_33 .........
Processing 26/186 AIDREAM_34 .........
Pr

In [18]:
# Next step is to add the pre_rt L1 and L3 labels to the affine processed directory :
# this depends on pre_RT_reference of the patient :
df_ref.groupby("pre_RT_reference").count()

,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_reference,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model
pre_RT_reference,,,,,,,,,,,,,
AC_repATLAS,119,119,119,119,119,119,119,119,119,117,119,119,119
AC_repATLAS native,1,1,1,1,1,1,1,1,1,1,1,1,1
GE_repNATIV,38,38,38,38,38,38,38,38,38,36,38,38,38
GE_repNativAXIALISEDnew,9,9,9,9,9,9,9,9,9,7,9,9,9
GE_repNativnew,12,12,12,12,12,12,12,12,12,12,12,12,12
"JBD : Quel repère ? Pas de crâne, jcomprends pas",1,1,1,1,1,1,1,1,1,1,1,1,1
Repère natif,1,1,1,1,1,1,1,1,1,1,1,1,1
skullstripaxialized RS recalé sur GE,5,5,5,5,5,5,5,5,5,5,5,5,5


In [ ]:
# First, for rep_ATLAS 